In [ ]:
import schedule
import time
import os
import subprocess

def run_notebook(notebook_name, background=False):
    print(f"Running: {notebook_name}")
    if background and notebook_name == "Revenue_generator.ipynb":
        subprocess.Popen(
            ["jupyter", "nbconvert", "--to", "notebook", "--execute", notebook_name, "--inplace"],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
    else:
        os.system(f"jupyter nbconvert --to notebook --execute {notebook_name} --inplace")

# Step 1: Run Stock_universe and trigger flow
def run_initial_and_start_flow():
    run_notebook("Stock_universe.ipynb")

    print("Running daily analysis notebooks after Stock_universe...")
    run_daily_notebooks()

    print("Running 40-min task immediately after daily notebooks...")
    chained_40min_tasks()

    start_scheduling_flow()

# Step 2: Manually run all daily notebooks once
def run_daily_notebooks():
    run_notebook("Properties.ipynb")

# Step 3: Start scheduling regular daily and 40-min flows
def start_scheduling_flow():
    schedule.every().day.at(f"00:01").do(run_notebook, "Properties.ipynb")

    schedule.every(40).minutes.do(chained_40min_tasks)
    print("Scheduling flow started.")

# 40-min task chain
def chained_40min_tasks():
    print("Starting 40-min chain...")
    run_notebook("Feature_Scalar.ipynb")
    run_notebook("Sarima_Network.ipynb")

    print("Generating CSV files...")
    with open("Money.csv", "w") as f:
        f.write("Wallet,Returns\n10000,0")
    with open("old_stock_data.csv", "w") as f:
        f.write("Stock,Quantity,Buy Price,Last Price\n")
    with open("return_statements.csv", "w") as f:
        f.write("Returns\n")

    # Run Revenue_generator in background
    run_notebook("Revenue_generator.ipynb", background=True)

# === Main ===
if __name__ == "__main__":
    run_initial_and_start_flow()

    while True:
        schedule.run_pending()
        time.sleep(1)
